In [ ]:
import pandas as pd
import warnings
import pandas as pd

import ee

import matplotlib.pyplot as plt
warnings.filterwarnings("ignore") 

ee.Initialize(project="teak-component-450514-d6")




In [1]:
from sklearn.neighbors import BallTree
import numpy as np

def spatial_thin_gdf(gdf, lat_col='Latitude', lon_col='Longitude', min_dist_km=5):
    # Convert coordinates to radians for haversine distance
    coords = np.radians(gdf[[lat_col, lon_col]].values)
    tree = BallTree(coords, metric='haversine')
    radius = min_dist_km / 6371.0  # 5 km in radians
    mask = np.ones(len(gdf), dtype=bool)
    for i in range(len(gdf)):
        if not mask[i]:
            continue
        ind = tree.query_radius([coords[i]], r=radius)[0]
        ind = ind[ind > i]
        mask[ind] = False
    return gdf[mask].reset_index(drop=True)

# Thinning for each group
beetles_thinned = spatial_thin_gdf(beetles_shp, lat_col='Latitude', lon_col='Longitude', min_dist_km=5)
butterflies_thinned = spatial_thin_gdf(butterflies_shp, lat_col='latitude', lon_col='longitude', min_dist_km=5)
dragonflies_thinned = spatial_thin_gdf(dragonflies_shp, lat_col='Latitude', lon_col='Longitude', min_dist_km=5)

thinned_gdfs = [beetles_thinned, butterflies_thinned, dragonflies_thinned]
thinned_names = ['Beetles', 'Butterflies and Moths', 'Dragonflies']
thinned_colors = ['red', 'blue', 'green']

for gdf, name, color in zip(thinned_gdfs, thinned_names, thinned_colors):
    n_points = len(gdf)
    ax = africa_shp.plot(color='lightgrey', figsize=(10, 10), edgecolor='black', alpha=0.5)
    gdf.plot(ax=ax, color=color, edgecolor='black', alpha=0.5, markersize=5)
    plt.title(f"{name} (5km thinned, n={n_points})", fontsize=16)
    plt.xlabel("Longitude", fontsize=14)
    plt.ylabel("Latitude", fontsize=14)
    plt.grid(True)
    plt.show()


NameError: name 'beetles_shp' is not defined

In [ ]:
for x in thinned_gdfs:
    print(x.shape)

In [ ]:
total_points = thinned_gdfs[0].shape[0] + thinned_gdfs[1].shape[0] + thinned_gdfs[2].shape[0]
print(f"Total thinned points across all groups: {total_points}")